In [4]:
import setGPU

import tensorflow as tf
import sys

sys.path.append('../scripts')

from data_reading import *

In [5]:
data_obj = public_datasets()

In [6]:
x_train, y_train, x_test, y_test = data_obj.load_mnist()
img_size = x_train.shape[1]
input_shape = (img_size, img_size, 1)
batch_size = 128
kernel_size = 3
filters = 16
latent_dim = 2
epochs = 3

In [7]:
#reparameterization trick
#instead of sampling from Q(z/x), sample eps = N(0, I)
#then z = z_mean+sqrt(var)*eps
def sampling(args):
    z_mean, z_log_var = args
    batch = tf.keras.backend.shape(z_mean)[0]
    dim = tf.keras.backend.int_shape(z_mean)[1]
    epsilon = tf.keras.backend.random_normal(shape=(batch,dim))
    return z_mean + tf.keras.backend.exp(0.5*z_log_var) * epsilon

In [8]:
#VAE model = encoder + decoder
#build encoder model
inputs = tf.keras.layers.Input(shape=input_shape, name='encoder_input')
x = inputs
for i in range(2):
    filters *=2
    x = tf.keras.layers.Conv2D(filters=filters, kernel_size=kernel_size, activation='relu', strides=2, padding='same')(x)
    
shape = tf.keras.backend.int_shape(x)

x = tf.keras.layers.Flatten()(x)
x = tf.keras.layers.Dense(16, activation='relu')(x)
z_mean = tf.keras.layers.Dense(latent_dim, name='z_mean')(x)
z_log_var = tf.keras.layers.Dense(latent_dim, name='z_log_var')(x)
z = tf.keras.layers.Lambda(sampling, output_shape=(latent_dim, ), name='z')([z_mean, z_log_var])

encoder = tf.keras.models.Model(inputs, [z_mean, z_log_var, z], name='encoder')
#encoder.summary()
#plot_model(encoder, to_file='../model/vae_mnist/vae_cnn_encoder.png', show_shapes=True)

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [9]:
#build decoder model
latent_inputs = tf.keras.layers.Input(shape=(latent_dim, ), name='z_sampling')
x = tf.keras.layers.Dense(shape[1]*shape[2]*shape[3], activation='relu')(latent_inputs)
x = tf.keras.layers.Reshape((shape[1], shape[2], shape[3]))(x)

for i in range(2):
    x = tf.keras.layers.Conv2DTranspose(filters=filters, kernel_size=kernel_size, activation='relu', strides=2, padding='same')(x)
    filters //= 2
    
outputs = tf.keras.layers.Conv2DTranspose(filters=1, kernel_size=kernel_size, activation='sigmoid', padding='same', name='decoder_output')(x)

decoder = tf.keras.models.Model(latent_inputs, outputs, name='decoder')
#decoder.summary()
#plot_model(decoder, to_file='../model/vae_mnist/vae_cnn_decoder.png', show_shapes=True)

outputs = decoder(encoder(inputs)[2])
vae = tf.keras.models.Model(inputs, outputs, name='vae')

In [10]:
models = (encoder, decoder)
data = (x_test, y_test)

val_mse = True

if val_mse:
    reconstruction_loss = tf.keras.losses.mse(tf.keras.backend.flatten(inputs), tf.keras.backend.flatten(outputs))
else:
    reconstruction_loss = tf.keras.losses.binary_crossentropy(tf.keras.backend.flatten(inputs), tf.keras.backend.flatten())
    
reconstruction_loss *= img_size*img_size
kl_loss = 1 + z_log_var - tf.keras.backend.square(z_mean) - tf.keras.backend.exp(z_log_var)
kl_loss = tf.keras.backend.sum(kl_loss, axis=-1)
kl_loss *= -0.5
vae_loss = tf.keras.backend.mean(reconstruction_loss + kl_loss)
vae.add_loss(vae_loss)
vae.compile(optimizer='rmsprop')
#vae.summary()
#plot_model(vae, to_file='../model/vae_mnist/vae_cnn.png', show_shapes=True)

vae.fit(x_train, epochs=epochs, batch_size=batch_size, validation_data=(x_test, None))

ValueError: Error when checking input: expected encoder_input to have 4 dimensions, but got array with shape (60000, 784)